In [12]:
import numpy as np
from mpi4py import MPI

Invalid MIT-MAGIC-COOKIE-1 key

# Monte-Carlo Integration 

Sample points

Estimate square by multiplying function value in each point by integration range(multiple rectangles)

Take mean from estimated rectangles


In [7]:
def MonteCarloIntegration(f, low, high, discretization):
    points = np.random.uniform(low, high, discretization)
    integal = (high-low)*sum(f(points))/len(points)
    return integal


In [8]:
def quadratic(x):
    return x**2

def linear(x):
    return x

In [9]:
MonteCarloIntegration(linear, 0, 1, 7000)

0.5018866093812954

In [11]:
MonteCarloIntegration(quadratic, -2, 2, 5000)

5.365658429262607

# Using MPI

In [14]:
def MPI_integration(discretization, func_type, low, high):

    FUNCS={
        "linear":linear,
        "quadratic":quadratic
    }

    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()

    step_size = (high-low)/size
    low_ = low + step_size*rank
    high_ = low + step_size*(rank+1)

    integral = MonteCarloIntegration(
        FUNCS[func_type],
        low_,
        high_,
        discretization
    )

    # sum will work because we have same sample points on each process
    result = comm.reduce(integral, op=MPI.SUM, root=0)

    if rank == 0:
        result /= size

        print(f"Integral of {func_type} function on range from {low} to {high} is {result}") 



In [3]:
!mpirun -np 2 --allow-run-as-root python lab6_mpi.py
     

Invalid MIT-MAGIC-COOKIE-1 key--------------------------------------------------------------------------
mpirun was unable to find the specified executable file, and therefore
did not launch the job.  This error was first reported for process
rank 0; it may have occurred for other processes as well.

NOTE: A common cause for this error is misspelling a mpirun command
      line parameter option (remember that mpirun interprets the first
      unrecognized command line token as the executable).

Node:       ledpc
Executable: python
--------------------------------------------------------------------------
2 total processes failed to start
